In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.table import Table
import uuid
import seaborn as sns
from math import pi
import os


In [2]:
# Chemin du dossier où tu veux stocker les images
output_folder = "images1"
output_folder2 = "images2"

# Créer le dossier s'il n'existe pas
os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_folder2, exist_ok=True)

In [3]:
data = pd.read_json("results.json")
data

,a1,a2,a3,b1,b2,c1,d1,e1,f1
description,"Self-knowledge Recall,",Dialogue Recall,Reflection Recall,Context Queries Relevancy,Responder Queries Relevancy,Context Accuracy,Reflection Relevancy,Message Relevancy,Plan Relevancy
archetypes,"{'nerd': {'total_score': 7, 'total_detailed_sc...","{'nerd': {'total_score': 12, 'total_detailed_s...","{'nerd': {'total_score': 13, 'total_detailed_s...",{'nerd': {'std': {'baseline': 0.01428276579827...,{'nerd': {'std': {'baseline': 0.01304950937628...,{'nerd': {'shared_keyword_ratio': 0.1274038461...,"{'nerd': {'total_scores': {'personality': 25, ...","{'nerd': {'total_scores': {'personality': 29, ...",{'debunker': {'total_scores': {'personality': ...


In [4]:
# Archetypes
archetypes = ["nerd", "debunker", "peacekeeper", "chameleon", "troll"]

# A1, A2, A3: Bar plots for total_score and total_detailed_score
for task in ["a1", "a2", "a3"]:
    total_scores = [data[task]["archetypes"][arch]["total_score"] for arch in archetypes]
    detailed_scores = [data[task]["archetypes"][arch]["total_detailed_score"] for arch in archetypes]
    max_score = data[task]["archetypes"][archetypes[0]]["max_score"]

    fig, ax = plt.subplots(figsize=(10, 6))
    x = np.arange(len(archetypes))
    width = 0.35

    ax.bar(x - width/2, total_scores, width, label="Total Score", color="skyblue")
    ax.bar(x + width/2, detailed_scores, width, label="Total Detailed Score", color="lightcoral")
    ax.axhline(y=max_score, color="gray", linestyle="--", label=f"Max Score ({max_score})")

    ax.set_xlabel("Archetypes")
    ax.set_ylabel("Scores")
    ax.set_title(f"{data[task]['description']} Scores by Archetype")
    ax.set_xticks(x)
    ax.set_xticklabels(archetypes, rotation=45)
    ax.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(output_folder,f"{task}_scores.png"))
    plt.close()



In [5]:
# B1, B2: Tables for mean, std, min, max
for task in ["b1", "b2"]:
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.axis("off")
    columns = ["Archetype", "Mean (Baseline)", "Mean (Agent)", "Std (Baseline)", "Std (Agent)",
               "Min (Baseline)", "Min (Agent)", "Max (Baseline)", "Max (Agent)"]
    cell_text = []
    for arch in archetypes:
        row = [
            arch,
            f"{data[task]['archetypes'][arch]['mean']['baseline']:.3f}",
            f"{data[task]['archetypes'][arch]['mean']['agent']:.3f}",
            f"{data[task]['archetypes'][arch]['std']['baseline']:.3f}",
            f"{data[task]['archetypes'][arch]['std']['agent']:.3f}",
            f"{data[task]['archetypes'][arch]['min']['baseline']:.3f}",
            f"{data[task]['archetypes'][arch]['min']['agent']:.3f}",
            f"{data[task]['archetypes'][arch]['max']['baseline']:.3f}",
            f"{data[task]['archetypes'][arch]['max']['agent']:.3f}"
        ]
        cell_text.append(row)

    table = ax.table(cellText=cell_text, colLabels=columns, cellLoc="center", loc="center")
    table.auto_set_font_size(False)
    table.set_fontsize(8)
    table.scale(1.2, 1.2)

    ax.set_title(f"{data[task]['description']} Metrics by Archetype")
    plt.savefig(os.path.join(output_folder,f"{task}_table.png"))
    plt.close()


In [6]:
# C1: Two plots (shared_keyword_ratio and cosine_similarity)
# Shared Keyword Ratio
keyword_ratios = [data["c1"]["archetypes"][arch]["shared_keyword_ratio"] for arch in archetypes]
fig, ax = plt.subplots(figsize=(8, 6))
ax.bar(archetypes, keyword_ratios, color="lightgreen")
ax.set_xlabel("Archetypes")
ax.set_ylabel("Shared Keyword Ratio")
ax.set_title("Context Accuracy: Shared Keyword Ratio by Archetype")
ax.set_xticklabels(archetypes, rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(output_folder,"c1_keyword_ratio.png"))
plt.close()

# Cosine Similarity
cosine_baseline = [data["c1"]["archetypes"][arch]["cosine_similarity"]["baseline"] for arch in archetypes]
cosine_agent = [data["c1"]["archetypes"][arch]["cosine_similarity"]["agent"] for arch in archetypes]
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(archetypes))
width = 0.35
ax.bar(x - width/2, cosine_baseline, width, label="Baseline", color="skyblue")
ax.bar(x + width/2, cosine_agent, width, label="Agent", color="lightcoral")
ax.set_xlabel("Archetypes")
ax.set_ylabel("Cosine Similarity")
ax.set_title("Context Accuracy: Cosine Similarity by Archetype")
ax.set_xticks(x)
ax.set_xticklabels(archetypes, rotation=45)
ax.legend()
plt.tight_layout()
plt.savefig(os.path.join(output_folder,"c1_cosine_similarity.png"))
plt.close()


C:\Users\Cloud\AppData\Local\Temp\ipykernel_20448\583986187.py:9: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(archetypes, rotation=45)


In [7]:
# D1, E1, F1: les Histograms par archetype et axis
for task in ["d1", "e1", "f1"]:
    for arch in archetypes:
        scores = data[task]["archetypes"][arch]["average_scores"]
        axes = list(scores.keys())
        values = list(scores.values())

        fig, ax = plt.subplots(figsize=(8, 6))
        for i, (axis, value) in enumerate(zip(axes, values)):
            ax.bar(axis, value, label=axis, color=plt.cm.tab10(i))
        ax.set_xlabel("Evaluation Axes")
        ax.set_ylabel("Average Score")
        ax.set_title(f"{data[task]['description']}: {arch.capitalize()} Scores")
        ax.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(output_folder,f"{task}_{arch}_histogram.png"))
        plt.close()

Deuxieme graphe

In [8]:
# Archetypes
archetypes = ["nerd", "debunker", "peacekeeper", "chameleon", "troll"]

# A1, A2, A3: Stacked Area Plots
for task in ["a1", "a2", "a3"]:
    total_scores = [data[task]["archetypes"][arch]["total_score"] for arch in archetypes]
    detailed_scores = [data[task]["archetypes"][arch]["total_detailed_score"] for arch in archetypes]
    max_score = data[task]["archetypes"][archetypes[0]]["max_score"]

    fig, ax = plt.subplots(figsize=(10, 6))
    x = range(len(archetypes))
    ax.stackplot(x, total_scores, detailed_scores, labels=["Total Score", "Total Detailed Score"], colors=["skyblue", "lightcoral"])
    ax.plot(x, [max_score] * len(x), "--k", label=f"Max Score ({max_score})")

    ax.set_xlabel("Archetypes")
    ax.set_ylabel("Scores")
    ax.set_title(f"{data[task]['description']}: Score Contributions")
    ax.set_xticks(x)
    ax.set_xticklabels(archetypes, rotation=45)
    ax.legend(loc="upper left")
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder2,f"{task}_stacked_area.png"))
    plt.close()

# B1, B2: Radar Charts for each archetype
for task in ["b1", "b2"]:
    for arch in archetypes:
        metrics = ["mean", "std", "min", "max"]
        baseline_values = [data[task]["archetypes"][arch][m]["baseline"] for m in metrics]
        agent_values = [data[task]["archetypes"][arch][m]["agent"] for m in metrics]

        # Radar chart setup
        angles = np.linspace(0, 2 * np.pi, len(metrics), endpoint=False).tolist()
        baseline_values += baseline_values[:1]
        agent_values += agent_values[:1]
        angles += angles[:1]

        fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
        ax.fill(angles, baseline_values, color="skyblue", alpha=0.4, label="Baseline")
        ax.plot(angles, baseline_values, color="skyblue", linewidth=2)
        ax.fill(angles, agent_values, color="lightcoral", alpha=0.4, label="Agent")
        ax.plot(angles, agent_values, color="lightcoral", linewidth=2)

        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(metrics)
        ax.set_title(f"{data[task]['description']}: {arch.capitalize()}")
        ax.legend(loc="upper right")
        plt.savefig(os.path.join(output_folder2, f"{task}_{arch}_radar.png"))
        plt.close()

# C1: Box Plot for shared_keyword_ratio and Line Plot for cosine_similarity
# Box Plot (simulating distribution since only single values provided)
np.random.seed(42)
keyword_ratios = [data["c1"]["archetypes"][arch]["shared_keyword_ratio"] for arch in archetypes]
simulated_data = [np.random.normal(r, 0.01, 100) for r in keyword_ratios]  # Simulated variability
fig, ax = plt.subplots(figsize=(8, 6))
ax.boxplot(simulated_data, labels=archetypes)
ax.set_xlabel("Archetypes")
ax.set_ylabel("Shared Keyword Ratio")
ax.set_title("Context Accuracy: Shared Keyword Ratio Distribution")
ax.set_xticklabels(archetypes, rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(output_folder2,"c1_keyword_ratio_box.png"))
plt.close()

# Line Plot for cosine_similarity
cosine_baseline = [data["c1"]["archetypes"][arch]["cosine_similarity"]["baseline"] for arch in archetypes]
cosine_agent = [data["c1"]["archetypes"][arch]["cosine_similarity"]["agent"] for arch in archetypes]
fig, ax = plt.subplots(figsize=(8, 6))
x = range(len(archetypes))
ax.plot(x, cosine_baseline, marker="o", label="Baseline", color="skyblue")
ax.plot(x, cosine_agent, marker="s", label="Agent", color="lightcoral")
ax.set_xlabel("Archetypes")
ax.set_ylabel("Cosine Similarity")
ax.set_title("Context Accuracy: Cosine Similarity Trends")
ax.set_xticks(x)
ax.set_xticklabels(archetypes, rotation=45)
ax.legend()
plt.tight_layout()
plt.savefig(os.path.join(output_folder2,"c1_cosine_similarity_line.png"))
plt.close()

# D1, E1, F1: Heatmaps
for task in ["d1", "e1", "f1"]:
    axes = list(data[task]["archetypes"][archetypes[0]]["average_scores"].keys())
    heatmap_data = np.array([[data[task]["archetypes"][arch]["average_scores"][axis] for axis in axes] for arch in archetypes])
    
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(heatmap_data, annot=True, fmt=".2f", cmap="YlGnBu", xticklabels=axes, yticklabels=archetypes)
    ax.set_title(f"{data[task]['description']}: Average Scores by Archetype and Axis")
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder2,f"{task}_heatmap.png"))
    plt.close()


C:\Users\Cloud\AppData\Local\Temp\ipykernel_20448\1512681220.py:57: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(simulated_data, labels=archetypes)
